# Explore and Download files from E-Periodica
used to investigate the api structures and download a small number of sample files

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

In [93]:
# url for a single journal
url = "https://www.e-periodica.ch/digbib/view?pid=cmt-001%3A1917%3A1%3A%3A4"
url

'https://www.e-periodica.ch/digbib/view?pid=cmt-001%3A1917%3A1%3A%3A4'

In [9]:
# get the html page 
resp = requests.get(url)

In [72]:
# parse the html text
soup = BeautifulSoup(resp.text)
print(soup.prettify()[:700], "\n\n...")

<!DOCTYPE html>
<html class="no-js viewer-page ep-page--viewer" lang="de">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <link href="/dist/css/main-2c0f6a9f99fa252623af3b27b0f6aadb.css;jsessionid=9471C6D7F922346C77DC2F453F1447E0" rel="stylesheet"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image 

...


In [74]:
# find the table of context table that lists the pages in the journal 
table_soup = soup.find('table',attrs={"class":"ep-view__toc__table","summary":"This table lists the magazines"})

In [75]:
# extract the attributes and put in a dataframe easy viewing
data = [row.attrs for row in table_soup.find_all("tr")]
index = range(len(data))
df = pd.DataFrame(data, index).dropna()

In [77]:
# get the filename from the data-imid field
df['filename'] = df['data-imid'].apply(lambda x: x.rsplit('!',maxsplit=1)[-1])

In [78]:
df

,id,data-pages,data-pid,data-imid,filename
1,n-2,1,4,cmt!1917_001!cmt-001_1917_001_0001.jpg,cmt-001_1917_001_0001.jpg
2,n-3,2,4,cmt!1917_001!cmt-001_1917_001_0001.jpg,cmt-001_1917_001_0001.jpg
3,n-4,3,5,cmt!1917_001!cmt-001_1917_001_0002.jpg,cmt-001_1917_001_0002.jpg
4,n-5,2,7,cmt!1917_001!cmt-001_1917_001_0004.jpg,cmt-001_1917_001_0004.jpg
5,n-6,2,8,cmt!1917_001!cmt-001_1917_001_0005.jpg,cmt-001_1917_001_0005.jpg
...,...,...,...,...,...
59,n-60,3,68,cmt!1917_001!cmt-001_1917_001_0065.jpg,cmt-001_1917_001_0065.jpg
60,n-61,3,70,cmt!1917_001!cmt-001_1917_001_0067.jpg,cmt-001_1917_001_0067.jpg
61,n-62,3,72,cmt!1917_001!cmt-001_1917_001_0069.jpg,cmt-001_1917_001_0069.jpg
62,n-63,2,74,cmt!1917_001!cmt-001_1917_001_0071.jpg,cmt-001_1917_001_0071.jpg


this is an example url for viewing an individual image with the iiif api  
user interface includes options for requesting different image resoutions/sizes

In [102]:
example_url =  "https://iiif.library.ethz.ch/iiif/2/e-periodica!cmt!1917_001!cmt-001_1917_001_0001.jpg/full/!1024,1024/0/default.jpg?response-content-disposition=attachment%3B%20filename%3Dcmt-001_1917_001_0001-1024.jpg"
print(example_url)


print("\n reconstructing the url from parts: \n")
img_width_height_options = {
                    "small":('1024','1024'),
                    "medium":('2048','2048'),
                    "large":('full','full')
                   }
im_width, im_height = img_width_height_options['small']
data_imid = "cmt!1917_001!cmt-001_1917_001_0001.jpg"
filename = "cmt-001_1917_001_0001"
file_format=".jpg"

im_url = "https://iiif.library.ethz.ch/iiif/2/e-periodica!{data_imid}/full/!{im_width},{im_height}/0/default.jpg?response-content-disposition=attachment%3B%20filename%3D{filename}-{im_width}{file_format}"
im_url = im_url.format(data_imid = data_imid,
                       im_width = im_width, 
                       im_height = im_height,
                       filename = filename,
                       file_format=file_format)
print(im_url)
print("\nurls are the same:", example_url==im_url)

https://iiif.library.ethz.ch/iiif/2/e-periodica!cmt!1917_001!cmt-001_1917_001_0001.jpg/full/!1024,1024/0/default.jpg?response-content-disposition=attachment%3B%20filename%3Dcmt-001_1917_001_0001-1024.jpg

 reconstructing the url from parts: 

https://iiif.library.ethz.ch/iiif/2/e-periodica!cmt!1917_001!cmt-001_1917_001_0001.jpg/full/!1024,1024/0/default.jpg?response-content-disposition=attachment%3B%20filename%3Dcmt-001_1917_001_0001-1024.jpg

urls are the same: True


In [66]:
# download a list of images from DataFrame

output_dir = "../data/raw/e-periodica/sample"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

total = df.shape[0]
for i, row in df.iterrows():
    
    print(f"\r downloading file {i} of {total}", end="")
    data_imid = row['data-imid']
    fname = row['filename']
    im_url = f"https://iiif.library.ethz.ch/iiif/2/e-periodica!{data_imid}/full/!1024,1024/0/default.jpg?response-content-disposition=attachment%3B%20filename%3D{fname}"
    
        response = requests.get(im_url)

    output_fpath = os.path.join(output_dir, fname)
    try:
        with open(output_fpath, "wb") as f:
            f.write(response.content)
    except Exception as E:
        print(E)        

    sleep(1)

 downloading file 63 of 63